# Multi-Taxi Routing — A* with an Admissible Heuristic


This solution Precomputes exact shortest travel-times (minutes) to each destination using Dijkstra and uses those values as an admissible heuristic h(n) for A*. Finds each taxi’s route with A* (f = g + h) where g is elapsed minutes and h is the Dijkstra time-to-goal.

Simulates taxis concurrently from t=0, enforcing the congestion rule: at most 2 taxis on an undirected edge; a 3rd taxi waits W minutes before retrying.
Output includes per-taxi route, any WAIT events, each taxi’s total time, total completion time and makespan.

**Cells**:
1. Implementation (Dijkstra heuristic + A* + concurrent simulator).
2. Sample input and run (expected output matches the problem statement).
3. `run_from_input_string()` helper.


In [3]:
from heapq import heappush, heappop
from math import sqrt
from typing import List, Tuple, Dict


def dijkstra_times(N:int, adj:Dict[int,List[Tuple[int,float]]], minutes_per_km:float, dst:int):
    INF = 1e18
    dist = [INF]*(N+1)
    dist[dst] = 0.0
    heap = [(0.0, dst)]
    while heap:
        d,u = heappop(heap)
        if d > dist[u] + 1e-12:
            continue
        for v, edge_km in adj[u]:
            w = edge_km * minutes_per_km
            if dist[u] + w + 1e-9 < dist[v]:
                dist[v] = dist[u] + w
                heappush(heap, (dist[v], v))
    return dist



def astar_path_with_heuristic(N:int, adj:Dict[int,List[Tuple[int,float]]], minutes_per_km:float, src:int, dst:int):
    h = dijkstra_times(N, adj, minutes_per_km, dst)
    INF = 1e18
    g = [INF]*(N+1)
    parent = [-1]*(N+1)
    g[src] = 0.0
    heap = [(g[src] + h[src], g[src], src)]
    closed = [False]*(N+1)
    while heap:
        f, gu, u = heappop(heap)
        if closed[u]:
            continue
        closed[u] = True
        if u == dst:
            break
        for v, edge_km in adj[u]:
            w = edge_km * minutes_per_km
            tentative = g[u] + w
            if tentative + 1e-9 < g[v]:
                g[v] = tentative
                parent[v] = u
                heappush(heap, (g[v] + h[v], g[v], v))
    if parent[dst] == -1 and src != dst and not closed[dst]:
        return []
    path = []
    cur = dst
    while cur != -1:
        path.append(cur)
        if cur == src:
            break
        cur = parent[cur]
    path.reverse()
    if len(path)==0 or path[0] != src:
        return []
    return path



def simulate_concurrent(N:int, edge_list: List[Tuple[int,int,float]], coords_list: List[Tuple[float,float]], trips: List[Tuple[int,int]], W_minutes: float, S_kmph: float):
    adj = {i: [] for i in range(1, N+1)}
    edge_dist = {}
    for u,v,d in edge_list:
        adj[u].append((v,d))
        adj[v].append((u,d))
        key = (min(u,v), max(u,v))
        edge_dist[key] = d
    minutes_per_km = 60.0 / S_kmph

    paths = []
    for (src,dst) in trips:
        path = astar_path_with_heuristic(N, adj, minutes_per_km, src, dst)
        paths.append(path)

    occupancy: Dict[Tuple[int,int], List[Tuple[float,float]]] = {k: [] for k in edge_dist.keys()}

    P = len(trips)
    pos_idx = [0]*P  
    ready_time = [0.0]*P
    finished = [False]*P
    waits = [[] for _ in range(P)]
    total_time = [0.0]*P

    EPS = 1e-9

    while not all(finished):

        min_time = min(ready_time[i] for i in range(P) if not finished[i])
        # collect batch with that ready_time (allow small eps)
        batch = [i for i in range(P) if not finished[i] and ready_time[i] <= min_time + EPS]
        # sort batch deterministically by taxi id
        batch.sort()
        progressed = False
        for i in batch:
            path = paths[i]
            if not path or pos_idx[i] >= len(path)-1:
                # already at destination
                finished[i] = True
                total_time[i] = ready_time[i]
                continue
            u = path[pos_idx[i]]
            v = path[pos_idx[i]+1]
            key = (min(u,v), max(u,v))
            d_km = edge_dist[key]
            edge_time = d_km * minutes_per_km
            # count occupancy at ready_time[i]
            count_on = 0
            for (s,e) in occupancy[key]:
                if s <= ready_time[i] + EPS and ready_time[i] < e - EPS:
                    count_on += 1
            if count_on >= 2:
                # must wait W_minutes at node
                ready_time[i] += W_minutes
                waits[i].append(((u,v), W_minutes))
                # will retry later; no progress now
            else:
                # can enter edge
                occupancy[key].append((ready_time[i], ready_time[i] + edge_time))
                # advance taxi along edge
                ready_time[i] = ready_time[i] + edge_time
                pos_idx[i] += 1
                progressed = True
                if pos_idx[i] >= len(path)-1:
                    finished[i] = True
                    total_time[i] = ready_time[i]
        # If none progressed in this batch, the loop will continue with updated ready_times
        # (this handles repeated waits)
        # continue while loop until all finished
        # small safeguard: if no one progressed and no waits occurred, break to avoid infinite loop
        # but in our logic either progress or waits must occur
    # Build results
    results = []
    for i in range(P):
        results.append({'route': paths[i], 'waits': waits[i], 'total_time': total_time[i]})
    total_completion = sum(r['total_time'] for r in results)
    makespan = max((r['total_time'] for r in results), default=0.0)
    return results, total_completion, makespan

def format_output(trips, results, total_completion, makespan):
    out_lines = []
    for i, (trip, res) in enumerate(zip(trips, results)):
        out_lines.append(f"Taxi {i+1}:")
        out_lines.append(f"Passenger {trip[0]}->{trip[1]}")
        out_lines.append("Route: " + " -> ".join(map(str, res['route'])))
        for (u_v, wt) in res['waits']:
            u,v = u_v
            out_lines.append(f"WAIT on edge ({u}->{v}): {wt:.1f} minutes")
        out_lines.append(f"Total time = {res['total_time']:.1f} minutes\n")
    out_lines.append(f"Total Completion Time = {total_completion:.1f} minutes")
    out_lines.append(f"(span = {makespan:.1f} minutes)")
    return "\n".join(out_lines)


In [4]:
# Sample Input (from the problem statement)
N = 8
M = 9
P = 3
W = 30.0
S = 40.0
coords = [
    (0,0),
    (30,0),
    (80,0),
    (30,60),
    (120,0),
    (30,120),
    (60,120),
    (110,120)
]
edges = [
    (1,2,30),
    (2,3,50),
    (2,4,60),
    (3,5,40),
    (4,6,70),
    (5,7,20),
    (6,7,30),
    (7,8,50),
    (3,6,90)
]
trips = [(2,7),(1,8),(3,4)]

results, total, makespan = simulate_concurrent(N, edges, coords, trips, W, S)
print(format_output(trips, results, total, makespan))


Taxi 1:
Passenger 2->7
Route: 2 -> 3 -> 5 -> 7
Total time = 165.0 minutes

Taxi 2:
Passenger 1->8
Route: 1 -> 2 -> 3 -> 5 -> 7 -> 8
WAIT on edge (2->3): 30.0 minutes
Total time = 315.0 minutes

Taxi 3:
Passenger 3->4
Route: 3 -> 2 -> 4
Total time = 165.0 minutes

Total Completion Time = 645.0 minutes
(span = 315.0 minutes)


In [5]:
# Utility to run from a pasted input string (same format as problem)

def run_from_input_string(s: str):
    tokens = s.strip().split()
    it = iter(tokens)
    N = int(next(it)); M = int(next(it)); P = int(next(it)); W = float(next(it)); S = float(next(it))
    coords = []
    for _ in range(N):
        x = float(next(it)); y = float(next(it)); coords.append((x,y))
    edges = []
    for _ in range(M):
        u = int(next(it)); v = int(next(it)); d = float(next(it)); edges.append((u,v,d))
    trips = []
    for _ in range(P):
        a = int(next(it)); b = int(next(it)); trips.append((a,b))
    res, total, makespan = simulate_concurrent(N, edges, coords, trips, W, S)
    print(format_output(trips, res, total, makespan))


In [6]:
run_from_input_string("""8 9 3 30 40
0 0
30 0
80 0
30 60
120 0
30 120
60 120
110 120
1 2 30
2 3 50
2 4 60
3 5 40
4 6 70
5 7 20
6 7 30
7 8 50
3 6 90
2 7
1 8
3 4""")

Taxi 1:
Passenger 2->7
Route: 2 -> 3 -> 5 -> 7
Total time = 165.0 minutes

Taxi 2:
Passenger 1->8
Route: 1 -> 2 -> 3 -> 5 -> 7 -> 8
WAIT on edge (2->3): 30.0 minutes
Total time = 315.0 minutes

Taxi 3:
Passenger 3->4
Route: 3 -> 2 -> 4
Total time = 165.0 minutes

Total Completion Time = 645.0 minutes
(span = 315.0 minutes)
